# QuaSaR: Identifying EEW Rings - Topology

[Quake Safe Rings](./1a_stations_faultlnes_plot.ipynb) - in our efforts to understand the station fault topology - we make use of the International Federation Data of Seismic Networks (FDSN), the global standard and a [data service](http://www.fdsn.org/services/) for sharing seismic sensor wave form data. The Obspy librarires support FDSN. The list of resources and services that are used for retrieving station inventory and waveform data.

In [ ]:
import glob
from obspy import read_inventory
from obspy.clients.fdsn import Client
from obspy.core import read, UTCDateTime
#from datetime import date

# Establish start and end time for retrieving waveform data
t_start = UTCDateTime.now()-518400 #6 days ago = 60s x 60m x 24h x 6d
t_end = UTCDateTime.now()+86400 #1 day in the future = 60s x 60m x 24h
print('Station startime: ', t_start, '\n & ending time: ', t_end)

try:
    #use either or GeoNet station service webservice URL or Obspy FDSN Client protocol to retrieve station data
    st_ws = 'https://service.geonet.org.nz/fdsnws/station/1/query?network=NZ&level=station&endafter=2020-12-31&format=xml'
    #st_ws = 'https://service.geonet.org.nz/fdsnws/station/1/query?network=NZ&station=CECS&level=channel'
    # Set FDSN client URL to GEONET short code
    client  = Client('GEONET')
    print("Client is",client)
except Exception as err:
    print("Error message:", err)


### Get all Station details

An initiatl step for object 1.A is determining the the types of operational seismic sensors and their locations. GoeNet hosts wave forms for a multitude of [sensor types](https://api.geonet.org.nz/network/sensor/type) (e.g. tidle guages, pressure gauges, seismometers, GNSS antennas, barometers, Microphones, Hydrophones and so on). The focus is on motion sensors of type: (i) accelerometer, (ii) broadband velocity, (iii) short period velocity, and (iv) GNSS. Furthermore, the sensors location code is unique to each sensor type. Therefore, one may chose to use the location code prefix or sensor type enumerator to select the desired sensors; i.e. seimograph and accelerometer stations. The motion sensors are used in both earthquake and volcanic seismic activity monitoring and early warning.

_Sensor types that are relevant to earthquake detection are:_
* 1 Accelerometer 
* 3 Broadband Seismometer 
* 4 GNSS Antenna 
* 8 Short Period Borehole Seismometer 
* 9 Short Period Seismometer 
* 10 Strong Motion Sensor

_Location codes reserved for the seismic sensors are:_
* 1? - weak motion sensors
* 2? - strong motion sensors

_Channel codes are:_ 

Defined in the GeoNet's [stream naming conventions](https://www.geonet.org.nz/data/supplementary/channels)
First letter of the code represents a combination of sampling rate and sensor bandwidth

First letter represts the sensor type 
* U (Ultra Long Period sampled at 0.01Hz, or SOH sampled at 0.01Hz)
* V (Very Long Period sampled at 0.1Hz, or SOH sampled at 0.1Hz)
* L (Broad band sampled at 1Hz, or SOH sampled at 1Hz)
* B (Broad band sampled at between 10 and 80 Hz, usually 10 or 50 Hz)
* S (Short-period sampled at between 10 and 80 Hz, usually 50 Hz)
* H (High Broad band sampled at or above 80Hz, generally 100 or 200 Hz)
* E (Extremely Short-period sampled at or above 80Hz, generally 100 Hz)

The second letter represents the sensor type, e.g.(listed are the ones relevant to seismometers)
* H (Weak motion sensor, e.g. measuring velocity)
* N (Strong motion sensor, e.g. measuring acceleration)
* L (Low gain sensor, usually velocity)
* M (Mass position, used for monitoring broadband sensors)

In [ ]:
''' All weak & strong motion, low gain, and mass possion sensor types '''
channels = "UH*,VH*,LH*,BH*,SH*,HH*,EH*,UN*,VN*,LN*,BN*,SN*,HN*,EN*"
st_type = {"UH" : "Weak motion sensor, e.g. measuring velocity\nUltra Long Period sampled at 0.01Hz, or SOH sampled at 0.01Hz",
           "VH" : "Weak motion sensor, e.g. measuring velocity\nVery Long Period sampled at 0.1Hz, or SOH sampled at 0.1Hz",
           "LH" : "Weak motion sensor, e.g. measuring velocity\nBroad band sampled at 1Hz, or SOH sampled at 1Hz",
           "BH" : "Weak motion sensor, e.g. measuring velocity\nBroad band sampled at between 10 and 80 Hz, usually 10 or 50 Hz",
           "SH" : "Weak motion sensor, e.g. measuring velocity\nShort-period sampled at between 10 and 80 Hz, usually 50 Hz", 
           "HH" : "Weak motion sensor, e.g. measuring velocity\nHigh Broad band sampled at or above 80Hz, generally 100 or 200 Hz",
           "EH" : "Weak motion sensor, e.g. measuring velocity\nExtremely Short-period sampled at or above 80Hz, generally 100 Hz",
           "UN" : "Strong motion sensor, e.g. measuring acceleration\nUltra Long Period sampled at 0.01Hz, or SOH sampled at 0.01Hz",
           "VN" : "Strong motion sensor, e.g. measuring acceleration\nVery Long Period sampled at 0.1Hz, or SOH sampled at 0.1Hz",
           "LN" : "Strong motion sensor, e.g. measuring acceleration\nBroad band sampled at 1Hz, or SOH sampled at 1Hz",
           "BN" : "Strong motion sensor, e.g. measuring acceleration\nBroad band sampled at between 10 and 80 Hz, usually 10 or 50 Hz",
           "SN" : "Strong motion sensor, e.g. measuring acceleration\nShort-period sampled at between 10 and 80 Hz, usually 50 Hz",
           "HN" : "Strong motion sensor, e.g. measuring acceleration\nHigh Broad band sampled at or above 80Hz, generally 100 or 200 Hz",
           "EN" : "Strong motion sensor, e.g. measuring acceleration\nExtremely Short-period sampled at or above 80Hz, generally 100 Hz"}

### Get list of stataions

Prepare an array of tuples necessary and sufficient station data:
* _station code_ as a unique identifier
* _coordinates_ longitude & latitude
* _elevation_ in meters above mean sea level

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

#plt.figure(figsize=(30, 40))
#fig, ax = plt.subplots(1,1,figsize=(30,40))

try:
    st_inv = client.get_stations(network='NZ', location="1?,2?", station='*', 
                             channel=channels, level='channel', starttime=t_start, endtime = t_end)

    ''' Print the station inventory '''
    print('Number of active stations:', len(st_inv[0].stations))
    st_type_dict = {"st_type":[]}
    st_loc_list = []
    st_inv_err_dict = []

    for each_st in range(len(st_inv[0].stations)):
        ''' use lat/lon paris only in and around NZ'''
        if(st_inv[0].stations[each_st].latitude < 0 and st_inv[0].stations[each_st].longitude > 0):
            each_st_type_dict = st_inv[0].stations[each_st].get_contents()
            ''' get the second character representing the station type '''
            st_type_dict["st_type"].append(each_st_type_dict["channels"][0][-3:-1])
            ''' list of corresponding station locations (lat / lon) '''
            st_loc_list.append([st_inv[0].stations[each_st].code, each_st_type_dict["channels"][0][-3:-1], st_inv[0][each_st].latitude, st_inv[0][each_st].longitude])
        else:
            st_inv_err_dict.append([st_inv[0].stations[each_st].code,st_inv[0][each_st].latitude, st_inv[0][each_st].longitude])

    print('Invalid stations:',st_inv_err_dict)
    print('Number of valid active stations:', len(st_loc_list))

    unique_st_types = set(st_type_dict["st_type"])
    '''
    core_samples_mask = np.zeros_like(st_type_dict["st_type"], dtype=bool)
    core_samples_mask[0:sum([len(x) for x in st_type_dict["st_type"]])] = True

    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_st_types))]
    '''
    # convert list to array for color coding the locations by station types
    '''
    legend_elements = []
    st_loc_arr = np.array(st_loc_list)
    for k, col in zip(unique_st_types, colors):
        class_member_mask = (np.array(st_type_dict["st_type"]) == k)
        xy = st_loc_arr[class_member_mask & core_samples_mask]
        print('Number of',str(st_type[k]),': %d' % len(xy))

        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=14, label=str(st_type[k]))
    
    plt.title('Estimated number of Stations: %d' % len(st_loc_arr), fontsize=40)
    plt.xlabel('Latitude', fontsize=30)
    plt.ylabel('Longitude', fontsize=30)
    plt.xticks(size=20)
    plt.yticks(size=20)
    plt.legend(loc='upper left', fontsize=20)
    plt.show()
    '''

except Exception as err:
    print("Error message:", err)

In [ ]:
'''
import numpy as np

st_coord = []

try:
    i = 0
    for i in range(len(st_inv[0].stations)):
        st_tuple = (st_inv[0][i].code, st_inv[0][i].latitude,st_inv[0][i].longitude, st_inv[0][i].elevation)
        st_coord.append(tuple(st_tuple))
        #st_coord = {'Code': st_inv[0][i].code,'Coordinates': {'Latitude': st_inv[0][i].latitude,'Longitude': st_inv[0][i].longitude},'Elavation': st_inv[0][i].elevation}
#        print(st_coord[i])

# Mapping stations
except Exception as err:
    print("Error message:", err)
'''

### Fault line coordinates

We have completed objective 1.A. However, we will also include a mapping of the fault lines to give a perception of the station distribution relative to that of the map of fault lines. 

In [ ]:
import matplotlib.pyplot as plt 
import json
from dictor import dictor

#plt.figure(figsize=(30, 40))

'''Extract nested values from a JSON tree.'''

try:
    with open('/home/nuwan/workspace/quasar/data/NZAFD/JSON/NZAFD_Oct_2020_WGS84.json') as json_file: 
        data = json.load(json_file)

    for each_feature in range(len(data['features'])):
        points = {"x":[], "y":[]}
        path = dictor(data,'features.{}.geometry.paths.0'.format(each_feature))
        for each_coordinate in range(len(path)):
            points["x"].append(path[each_coordinate][0])
            points["y"].append(path[each_coordinate][1])

except Exception as err:
    print("Error message:", err)

## OBJECTIVE 1.A - STATION CLUSTERS

### Cluster Stations

Apply DBSCAN to cluster stations with an epsilon < 30Km. DBSCAN is preferred over K-means clustering because K-means clustering considance the variance while DBSCAN considers a distance function. It gives the capacity to build clusters serving the criteria of < 30Km distance between stations.

#### Data clensing

#### Cluster
1. Apply DBSCAN
   1. Inherent __problem of DBSCAN__ is that it characterises data points to be in the same clusted if pair-wise data points satisfy the epsilon condition. This would not adequately satisfy the required condition that all data points in a a cluster are within the desired epsilon distance.
1. Compute the cluster property measures to estimate the acceptability
1. Dump the output to a file including cluster label, lat/lon, station code, and so on

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
import sklearn.utils
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs

def get_dbscan_labels(st_arr):
    err="0"
    try:
        X, labels_true = make_blobs(n_samples=len(st_arr), centers=st_arr, cluster_std=0.4,random_state=0)
        db = DBSCAN(eps=30.0/6371.0, min_samples=3, algorithm='ball_tree', metric='haversine').fit(np.radians(X))
        print('DBSCAN epsilon:',db.eps,'algorithm:', db.algorithm, 'metric: ', db.metric)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
#        print('core samples mask', len(core_samples_mask),core_samples_mask)
        labels = db.labels_
#        print("DBSCAN found %0.3f labels" % labels )
    except Exception as err:
        print("Error message:", err)
        labels = ""
    return labels, labels_true, core_samples_mask

### Parametric analysis of the clusters

A large propotion of the sensors could be clustered to have, at least, 03 senors in a cluster and that they are < 30Km distance from each other; which also is the basis for the ```eps = 30.0/6371.0``` (epsilon convereted to radians using the length of Earth's radius 6371Km). The estimated number of _noise points_ tell us the number of sensors that didn't belong to any cluster. The particular geodedic data cannot be clustered with KD-Trees or any Tree algorithm. However, chosing ```algorithm = "ball_tree"``` is recommended as it is well suited for geospatial data clustering. The ```metric = "haversine"``` is naturally required to calculate the distance between two geographical points. Finally, the ```st_dbscan_arr``` comprising latitude and longitude decimal data is converted to radians to be consistent with using the _haversine_ distance funcation.

In [ ]:
# Remove all columns except the lat / lon pairs
tmp_st_coord = np.delete(st_coord, [0,3],axis=1).astype(np.float)
# Remove bad coordinates specific to NZ
tmp_clean_st_coord = [item for item in tmp_st_coord if item[0] < 0 and item[1] > 0]
# Convert to float to avoid throwing a datatype error in the plot function
station_coordinates = np.array(tmp_clean_st_coord).astype(np.float)
#print('Station coordinates:\n',' '.join([str(elem) for elem in tmp_nolabel_arr])) 

# Run dbscan and get the labels
labels, labels_true, core_samples_mask = get_dbscan_labels(station_coordinates)
#print('core samples mask', len(core_samples_mask),core_samples_mask)

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Total number of stations: %d' % len(labels))
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

# Plot a histogram of the station distribution

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print(f"Adjusted Mutual Information: %0.3f" % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(station_coordinates, labels))

#### Plot results
1. plot clusters with varied colors unique to each cluster
1. plot fault lines to show closes sensor in cluster to the fault line

In [ ]:
# #############################################################################
# Plot result
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

plt.figure(figsize=(30, 40))
#nz_map = Basemap(width=15000,height=15000,projection='merc',
#            resolution='l',lat_0=-40,lon_0=176.)
#nz_map.drawcoastlines()

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = station_coordinates[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    # uncomment to plot the noise
    #xy = station_coordinates[class_member_mask & ~core_samples_mask]
    #plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
    #         markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.legend(loc='upper left', fontsize=20)
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.show()

### Mean Nearest Neighbour Distance Statistics

Compute the mean distance between nearest neigbours of a minimum 3 points
* https://scikit-learn.org/stable/modules/neighbors.html
* https://pysal.org/notebooks/explore/pointpats/distance_statistics.html#Mean-Nearest-Neighbor-Distance-Statistics

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Augment station array with cluster number
# Start a new station coorinates and details tuple
st_list = []
i=0
for i in range(len(labels)):
    st_row = [tmp_arr[i,0],labels[i],tmp_arr[i,1],tmp_arr[i,2],tmp_arr[i,3]]
    st_list.append(list(st_row))

clusters = list({item[1] for item in st_list})

for each_cluster in clusters:
    cluster_list = list(st_list[j] for j in range(len(st_list)) if st_list[j][1] == each_cluster)
    cluster_arr = np.delete(cluster_list, [0,1,4],axis=1).astype(np.float)
    nbrs = NearestNeighbors(n_neighbors=3, algorithm='brute', metric='haversine').fit(cluster_arr)
    distances, indices = nbrs.kneighbors(cluster_arr)
    print(nbrs.kneighbors_graph(cluster_arr).toarray())
    
    each_cluster_clique = client.get_stations(latitude=-42.693,longitude=173.022,maxradius=30.0/6371.0, starttime = "2016-11-13 11:05:00.000",endtime = "2016-11-14 11:00:00.000")
    print(each_cluster_clique)
    _=inventory.plot(projection="local")
    
    break

sorted_rank = sorted(st_list, key=lambda i: (int(i[1])), reverse=True)
#print('Code, Cluster, Latitude, Longitude, Elevation')
#print(sorted_rank)

### Discussion of DBSCAN results
It is evident from the cluster with large volume of data points are spread across the geography. Therefore, DBSCAN is shown to be innopriate for clustering stations to estimate whether they hold the property of being 30Km within each other.

Next we 

## RESOURCES
1. [Global data services and standards](http://www.fdsn.org/services/) offered by the International Federation Data of Seismic Networks (FDSN). 
1. GEONET resources:
   1. [Stream Naming Conventions](https://www.geonet.org.nz/data/supplementary/channels) are based on historical usage together with recommendations from the [SEED manual](https://www.fdsn.org/seed_manual/SEEDManual_V2.4.pdf)
   1. [Python tutorials](https://www.geonet.org.nz/data/tools/Tutorials) for using GeoNet resources
1. [Seismo-Live](https://krischer.github.io/seismo_live_build/html/Workshops/2017_Baku_STCU_IRIS_ObsPy_course/07_Basic_Processing_Exercise_solution_wrapper.html) examples of get station waveform, inventory, event, arrival time, response, and plotting using obspy
1. Choosing [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) over KMeans: 
   1. Fundermentally KMeans requires us to first select the number of clusters we wish to find and DBSCAN doesn't.
   1. [clustering to reduce spatial data sizes](https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/) KMeans is not an ideal algorithm for latitude-longitude spatial data because it minimizes variance, not geodetic distance. 
   1. [Explanation of DBSCAN clustering](https://towardsdatascience.com/explaining-dbscan-clustering-18eaf5c83b31) also identifies a drawback of KMeans clustering as it is vulnerable to outliers and outliers have a significant impact on the way the centroids moves.
1. [Example of scikit-learn DBSCAN](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html)
1. [obspy.geodetics](https://docs.obspy.org/packages/obspy.geodetics.html) - various geodetic utilities for ObsPy - try an alternative clustering method with obspy geodetics
1. Mapping tutorials
   1. Visualization: [Mapping Global Earthquake Activity](http://introtopython.org/visualization_earthquakes.html)
   1. Plotting data on a map [(Example Gallery)](https://matplotlib.org/basemap/users/examples.html)